In [1]:
import pandas
import numpy
import os
import sys
import json
from erlab_coat.preprocessing import *
from erlab_coat.meta import *

In [2]:
path = './'

In [3]:
collection = parse_erlab_covid19_glance_collection(path)

/Users/mednet_machine/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
#def preprocess_collection(collection):


In [5]:
## preprocessing
for key in preprocessings.keys():
    for table_name in preprocessings[key].keys():
        table = collection[key][table_name].copy()
        lambdas = preprocessings[key][table_name]['lambdas']
        renames = preprocessings[key][table_name]['rename']
        remove = preprocessings[key][table_name]['remove']
        for column in lambdas.keys():
            table[column] = table[column].apply(lambdas[column])
        table.rename(renames, inplace=True, axis="columns", errors="raise")
        table.drop(columns=remove, inplace=True)
        collection[key][table_name] = table.copy()
        table = None

In [6]:
# income
def income_check1(x):
    try:
        output = x.split(',')[1]
    except:
        output = 'bad'
    return output

collection['county']['diversityindex']['state'] = collection['county']['diversityindex']['Location'].apply(
    income_check1)
collection['county']['diversityindex'] = collection['county']['diversityindex'][
    collection['county']['diversityindex']['state'] != 'bad']
collection['county']['diversityindex']['county'] = collection['county']['diversityindex']['Location'].apply(
    lambda x: remove_county_word(x.split(', ')[0]))
collection['county']['diversityindex'].drop(columns=['Location'], inplace=True)

# diversity
collection['county']['diversityindex']['state'] = collection['county']['diversityindex']['state'].apply(
    lambda x: x.strip())

# mortality
def mortality_check1(x):
    try:
        output = x.split(', ')[1]
    except:
        output = 'bad'
    return output

collection['county']['mortality']['state'] = collection['county']['mortality']['county'].apply(
    mortality_check1).copy()
collection['county']['mortality'] = collection['county']['mortality'][
    collection['county']['mortality']['state'] != 'bad']
collection['county']['mortality']['county'] = collection['county']['mortality']['county'].apply(
    lambda x: remove_county_word(x.split(',')[0]))
collection['county']['mortality']['state'] = collection['county']['mortality']['state'].apply(
    lambda x: state_abbreviations[x])

# election
collection['county']['election'].drop(columns=['state_po'], inplace=True)
collection['county']['election'] = collection['county']['election'][
    collection['county']['election']['year'] == 2016]
collection['county']['election']['state'] = collection['county']['election']['state'].apply(
    lambda x: state_abbreviations[x])
output = {
    'state': [],
    'county': [],
    'democrat': [],
    'republican': [],
    'other': [],
}

for i in range(collection['county']['election'].shape[0]):
    row = collection['county']['election'].iloc[i, :]
    output['state'] += [row['state']]
    output['county'] += [row['county']]
    if row['party'] == 'democrat':
        output['democrat'] += [row['candidatevotes']]
        output['republican'] += [0]
        output['other'] += [0]
    elif row['party'] == 'republican':
        output['republican'] += [row['candidatevotes']]
        output['democrat'] += [0]
        output['other'] += [0]
    else:
        output['republican'] += [0]
        output['democrat'] += [0]
        output['other'] += [row['candidatevotes']]

collection['county']['election'] = pandas.DataFrame(output)

## icu beds
collection['county']['icu_beds']['state'] = collection['county']['icu_beds']['state'].apply(
    lambda x: state_abbreviations[x])

## income [removed]
# collection['county']['income'] = collection['county']['income'].groupby(
#     ['county', 'state']).mean().reset_index().copy()

## land and water
collection['county']['land_and_water'] = collection['county']['land_and_water'].loc[:,
                                         ['state', 'county', 'ALAND', 'AWATER', 'ALAND_SQMI', 'AWATER_SQMI']]

In [7]:
mortality = collection['county']['mortality'].copy().drop(columns=[
    'Mortality Rate, 1980*',
    'Mortality Rate, 1980* (Min)',
    'Mortality Rate, 1980* (Max)',
    'Mortality Rate, 1985*',
    'Mortality Rate, 1985* (Min)',
    'Mortality Rate, 1985* (Max)',
    'Mortality Rate, 1990*',
    'Mortality Rate, 1990* (Min)',
    'Mortality Rate, 1990* (Max)',
    'Mortality Rate, 1995*',
    'Mortality Rate, 1995* (Min)',
    'Mortality Rate, 1995* (Max)',
    'Mortality Rate, 2000*',
    'Mortality Rate, 2000* (Min)',
    'Mortality Rate, 2000* (Max)',
    'Mortality Rate, 2005*',
    'Mortality Rate, 2005* (Min)',
    'Mortality Rate, 2005* (Max)',
    'Mortality Rate, 2010*',
    'Mortality Rate, 2010* (Min)',
    'Mortality Rate, 2010* (Max)'
]).rename({
    'Mortality Rate, 2014*': 'mortality_rate',
    'Mortality Rate, 2014* (Min)': 'min_mortality_rate',
    'Mortality Rate, 2014* (Max)': 'max_mortality_rate',
    '% Change in Mortality Rate, 1980-2014': "change_in_mortality_rate",
    '% Change in Mortality Rate, 1980-2014 (Min)': "min_change_in_mortality_rate",
    '% Change in Mortality Rate, 1980-2014 (Max)': "max_change_in_mortality_rate"
}, inplace=False, axis="columns", errors="raise")
mortality = mortality.groupby(['county', 'state']).mean().copy()

#todo: shady
# census
census_full = collection['county']['census_full'].copy()
census_full = census_full.groupby(['county', 'state']).mean()

land_and_water = collection['county']['land_and_water'].copy()
land_and_water = land_and_water.groupby(['county', 'state']).sum()

election = collection['county']['election'].copy()
election = election.groupby(['county', 'state']).sum()

icu_beds = collection['county']['icu_beds'].copy()
icu_beds = icu_beds.groupby(['county', 'state']).sum()

diversity = collection['county']['diversityindex'].copy()
diversity = diversity.groupby(['county', 'state']).sum()

# dataframes = [aggregate_covid_county_table, census_full, icu_beds, diversity, election, land_and_water,
#               aggregate_income_by_county, mortality]

In [8]:
#census_full, icu_beds, diversity, election, land_and_water, mortality

In [9]:
diversity.columns

Index(['diversity_index', 'diversity_black_race', 'diversity_native_race',
       'diversity_asian_race', 'diversity_pacific_race',
       'diversity_two_or_more_races', 'diversity_hispanic_race',
       'diversity_white_race'],
      dtype='object')

In [10]:
collection['county']['cases'].columns

Index(['confirmed_date', 'county', 'state', 'confirmed_count', 'death_count',
       'recovered_count'],
      dtype='object')

In [12]:
covid_hospitalizations_df, influenza_activity_level_df = get_cdc_data(os.path.join(path, 'resolution/state/cdc_covid')
                                                                    )

In [13]:
influenza_activity_level_df

,state,state_infleunza_activity_level,confirmed_date,day_of_the_year,location
0,AL,12.0,2020-01-04,3,AL
1,AK,7.0,2020-01-04,3,AK
2,AZ,10.0,2020-01-04,3,AZ
3,AR,8.0,2020-01-04,3,AR
4,CA,10.0,2020-01-04,3,CA
...,...,...,...,...,...
860,WV,1.0,2020-01-19,18,WV
861,WI,7.0,2020-01-19,18,WI
862,WY,1.0,2020-01-19,18,WY
864,PR,6.0,2020-01-19,18,PR


In [15]:
covid_hospitalizations_df.columns

Index(['state', 'confirmed_date',
       'state_cumulative_covid_hospitalization_rate_per_100k_ag0_4',
       'state_weekly_covid_hospitalization_rate_per_100k_ag0_4',
       'day_of_the_year', 'location',
       'state_cumulative_covid_hospitalization_rate_per_100k_ag18_49',
       'state_weekly_covid_hospitalization_rate_per_100k_ag18_49',
       'state_cumulative_covid_hospitalization_rate_per_100k_ag5_17',
       'state_weekly_covid_hospitalization_rate_per_100k_ag5_17',
       'state_cumulative_covid_hospitalization_rate_per_100k_ag50_64',
       'state_weekly_covid_hospitalization_rate_per_100k_ag50_64',
       'state_cumulative_covid_hospitalization_rate_per_100k_ag65p',
       'state_weekly_covid_hospitalization_rate_per_100k_ag65p',
       'state_cumulative_covid_hospitalization_rate_per_100k_ag65_74',
       'state_weekly_covid_hospitalization_rate_per_100k_ag65_74',
       'state_cumulative_covid_hospitalization_rate_per_100k_ag75_84',
       'state_weekly_covid_hospitalizat